## 1. Kütüphanelerin Yüklenmesi

Projemizde kullanacağımız temel Python kütüphanelerini yüklüyoruz.
#### Numpy
NumPy bilimsel hesaplamaların hızlı bir şekilde yapılabilmesini sağlayan bir matematik kütüphanesidir. Çok boyutlu dizileri ve matrisleri destekler.
#### Pandas
Veri işleme ve analizi için sıklıkla kullanılan bir kütüphanedir. Verinin tablo yapısında işlenmesine olanak tanır.

#### Matplotlib
Veri görselleştirme için kullanılan temel Python kütüphanesidir, grafik oluşturmayı ve bu grafikler üzerinde farklı görselleştirmeler yapmayı sağlar.

#### Seaborn
Matplotlib tabanlı istatistiksel bir veri görselleştirme kütüphanesidir. Estetik ve anlamlı grafikler elde etmeyi sağlar.

Ayrıca kod sayfamızın temiz gözükmesi için uyarıları gizledik.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 2. Veri Setinin Yüklenmesi

Veri setimizi Kaggle ortamına dahil ediyoruz. Veri seti iki ayrı dosya halinde verilmiştir: "Fake.csv" ve "True.csv".  
Bu dosyaları okuyup, her birine **"label"** sütunu ekleyerek tek bir veri çerçevesi haline getiriyoruz. Verileri, tablo yapısında elde edebilmek için; **"pd"** (Pandas) kütüphanesini kullanarak **"df"** isimli DataFrame'e atadık.


In [ ]:
fake = pd.read_csv("/kaggle/input/real-and-fake-news/Fake.csv")
real = pd.read_csv("/kaggle/input/real-and-fake-news/True.csv")

fake['label'] = 1  #fake haberler = 1
real['label'] = 0  #gerçek haberler = 0

df = pd.concat([fake, real], ignore_index=True) #birleştirme işlemi
df.head()

## 3. Veri Setinin Karıştırılması (Shuffle)

Fake ve Real haberler iki farklı dosyada tutulduğu için, veri seti birleştirildiğinde sıralı (önce tüm fake haberler, sonra tüm real haberler) hale gelmiştir.  
Bu durum, eğitim ve test ayrımında modelin tüm etiketleri tek sınıftan öğrenmesine neden olabilir. **"sample()"** metodunu kullanarak veri setini karıştırıyoruz.

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True) #veri setini karıştır
df.head()

## 4. Veri Setinin Genel Yapısı
Veri setinin yapısını anlamak için **"info()"** metodunu kullanıyoruz. Veri setimiz 44898 satır ve 5 sütundan oluşuyor. Bu kümedeki 4 değişkeni tahminleyici olarak, 1 değişkeni ise hedef değişken olarak kullanacağız. Hedef değişken olarak "label" sütununu seçiyoruz çünkü haberlerin gerçek mi fake mi olduğu bilgisi bu sütunda bulunuyor.

Kütüphanelerimizin doğru şekilde çalışabilmesi için verinin tamamının non-null ve aynı veri tipinde olması gerekmektedir, bunu kontrol edelim:

"Dtype" bilgisinde görüldüğü üzere tüm veriler aynı veri tipinde. **"isnull()"** ile null veri olmadığını teyit ediyoruz.

In [ ]:
df.info()
print("\nSatır sayısı:", df.shape[0])
print("Sütun sayısı:", df.shape[1])
df.isnull().sum()

## 5. Etiket (label) Sütununun Dağılımı

Hedef değişkenimiz olan "**label**" sütunundaki, fake/real haberlerin sayısını matplotlib kütüphanesi aracılığıyla grafik görünümünde inceliyoruz:

23481 adet fake, 21417 adet real haber olduğunu gözlemledik; oran farkı çok fazla değil yani dağılım dengeli. Dağılımın dengeli olması modelin taraflı olma riskini azaltır. 

In [ ]:
sns.countplot(data=df, x='label')
plt.title("Etiket Dağılımı (Gerçek vs Sahte Haber)")
plt.xlabel("Label (0 = Real, 1 = Fake)")
plt.ylabel("Adet")
plt.show()

df['label'].value_counts()

## 6. Veriyi Anlama (Haber Kategorilerinin Dağılımı)

Veri setindeki haberlerin hangi kategorilere ait olduğunu analiz ediyoruz. Veri setinde haber konuları çeşitlilik gösteriyor, **"subject"** sütunu modeli beslemek için anlamlı olabilir. Bazı konular sahte habere daha yatkın olabilir bu nedenle bu sütunu sayısallaştırarak modele dahil etmek mantıklı olacaktır.

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='subject', order=df['subject'].value_counts().index)
plt.xticks(rotation=45)
plt.title("Haber Konularının Dağılımı")
plt.xlabel("Konu")
plt.ylabel("Haber Sayısı")
plt.show()

df['subject'].value_counts()

## 7. Metin Uzunluklarının Dağılımı

Haber metinlerinin uzunluklarını inceleyerek fake/real haberler arasında fark olup olmadığına bakıyoruz:

Ortalama uzunluk: 2.469 karakter                           
Maksimum: 51.794 karakter (aşırı uzun bir haber)                    
Minimum: 1 karakter (muhtemelen boş ya da anlamsız içerik — bunu filtrelememiz gerekebilir)              
Dağılım sağa çarpık (çoğu haber kısa, bazıları aşırı uzun)

In [ ]:
df['text_length'] = df['text'].apply(len)

plt.figure(figsize=(10,5))
sns.histplot(data=df, x='text_length', hue='label', bins=50, kde=True)
plt.title('Haber Metni Uzunluğu Dağılımı (Label Bazlı)')
plt.xlabel('Metin Uzunluğu (Karakter)')
plt.ylabel('Haber Sayısı')
plt.show()

df['text_length'].describe()

## 8. Veri Ön İşleme: Gürültülü Verilerin Temizlenmesi

Bir önceki adımda ("text_length" özelliği ile) gerçekleştirdiğimiz analizde, bazı haberlerin sadece 1 karakter uzunluğunda olduğunu gözlemlemiştik. 
Bu tarz veriler model için anlamsız ve yanıltıcı olabileceğinden, bu haberleri veri setinden çıkarıyoruz.

In [ ]:
df = df[df['text'].apply(len) > 10]  #10 karakterden kısa olan haberleri temizliyoruz
df.reset_index(drop=True, inplace=True)
print("Yeni veri seti boyutu:", df.shape)

## 9. Kategorik Verilerin Sayısallaştırılması (One-Hot Encoding)

Makine öğrenmesi algoritmaları kategorik (yazı tipi) verilerle doğrudan çalışamaz. Bu nedenle "**subject**" sütununu modelin anlayabileceği şekilde One-Hot Encoding yöntemiyle sayısal formata dönüştürüyoruz.
Bu işlemi "get_dummies()" metodu aracılığıyla gerçekleştiriyoruz. Bu metot her bir kategorik değeri ayrı bir sütun haline getirip 0 ve 1'lerle temsil etmeyi sağlar.

Ayrıca "drop_first=True" kullanarak kategorinin ilk değerini atıyoruz, böylece veri setinde gereksiz fazlalık oluşmasını önlemiş oluyoruz.

In [ ]:
df = pd.get_dummies(df, columns=['subject'], drop_first=True)
df.head()

## 10. Haber Metinlerinin Sayısallaştırılması (TF-IDF)
Veri setimizdeki **"text"** ve **"title"** sütunları, haberlerin metinsel içeriğini içermektedir. Makine öğrenmesi algoritmaları metinleri doğrudan anlayamadığından metinleri sayısal verilere çevirmemiz gerekir. Bu işlem için yeni bir kütüphane kullanacağız.

#### Scikit-learn (sklearn):
Makine öğrenmesi modelleri, veri ön işleme (TF-IDF, encoding), veri bölme (train_test_split) ve model değerlendirme işlemleri için kullanılır. **"TfidfVectorizer (from sklearn)"** metin verilerini sayısal vektörlere dönüştürmek için kullanılan özel bir sınıftır.

### TF-IDF Nedir?
TF-IDF (Term Frequency - Inverse Document Frequency), bir kelimenin bir belge içerisindeki önemini ölçen bir tekniktir.

#### TF (Term Frequency): 
Kelimenin ilgili belge içinde kaç kez geçtiği.          
#### IDF (Inverse Document Frequency):
Kelimenin tüm belgelerdeki yaygınlığına göre ağırlık düşürme.

Bu yöntem sayesinde anlamsız veya çok sık geçen kelimeler (örn: "ve", "bu", "bir") daha az etkili olur, önemli kelimeler öne çıkar.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#text için TF-IDF
tfidf_text = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=5000)
X_text = tfidf_text.fit_transform(df['text'])

#title için TF-IDF
tfidf_title = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=1000)
X_title = tfidf_title.fit_transform(df['title'])

## 11. Özellik Seçimi ve Hazırlık

Veri setindeki *"date"* sütunu ile model genelleme yapmakta zorlanabilir bu nedenle modelde kullanmayacağız.  
*"text_length"* sütununun sınıflar arasında yeterince ayırt edici olmadığını düşündüğümüz için kullanmaktan vazgeçtik.

Modelde kullanılacak ana özellikler:
- **"text"** (TF-IDF, max_features=5000)
- **"title"** (TF-IDF, max_features=1000)
- **"subject"** (Bu sütunu ekleyip eklemeyeceğimize deneyip sonuçlara bakarak karar vereceğiz.)


In [ ]:
df = df.drop(['date', 'text_length'], axis=1) #gereksiz sütunları kaldırdık

## 12.1. Özellik Birleştirme – Versiyon 1 (Subject Sütunu Dahil Değil)

Bu versiyonda modele yalnızca haberin metni (text) ve başlığı (title) TF-IDF yöntemiyle dönüştürülerek dahil ettik.
"subject" sütunu, modelin performansına etkisi ayrıca test edileceği için bu versiyonda bilinçli olarak dışarıda bıraktık.

TF-IDF işlemi sonucu oluşan X_text ve X_title matrisleri; normal pandas DataFrame değil, sparse matrix (seyrek matris) formatındadır. Bu format, büyük boyutlu metin verisini daha verimli saklamak için kullanılır.
**"hstack()"** metodu aracılığıyla yatay (horizontal) olarak iki sparse matrisi yan yana ekledik.

In [ ]:
from scipy.sparse import hstack

#subject hariç: sadece metin ve başlık
X = hstack([X_text, X_title]) #iki matrisi birleştirdik (giriş verisi)
y = df['label'] #modelin tahmin etmeye çalışacağı hedef değişken (iki versiyon için de aynı)

## 12.2 Özellik Birleştirme – Versiyon 2 (Subject Sütunu Dahil)

Bu versiyonda modele hem "text" ve "title" (TF-IDF ile), hem de "subject" sütununu (one-hot encoding sonrası) dahil ettik.         
Amaç: Haberlerin konularının (subject) sınıflandırma başarısına katkısı olup olmadığını test etmek.

"subject" sütunu, one-hot encoding işleminden sonra çok sayıda 0 ve 1 değerinden oluşan yeni sütunlara ayrılmıştır. Ancak TF-IDF ile dönüştürülmüş "text" ve "title" sütunları, genellikle küçük değerler (örneğin 0.003 gibi) içerir.     
Bu fark, modelin bazı sütunları fazla önemsemesine neden olabilir. Bu nedenle "subject" ile ilgili sütunları, modelin diğer girişleriyle *aynı ölçekte* olması için ölçeklendirdik. 

Burada **"StandardScaler"** sınıfı kullanılarak bu sütunlar standart sapmalarına göre normalize ettik. *"with_mean=False"* parametresi ile sparse matris yapısına uygun hale getiriyoruz.

In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix

#subject dışındaki sayısallaştırılmış sütunları aldık
subject_cols = df.drop(['text', 'title', 'label'], axis=1).values

scaler = StandardScaler(with_mean=False)
subject_scaled = scaler.fit_transform(subject_cols) #ölçekle

X_with_subject = hstack([X_text, X_title, csr_matrix(subject_scaled)]) #birleştir


## 13. Eğitim ve Test Verisinin Ayrılması

Model eğitimi için, özellikler (*X*) ve hedef değişken (*y*) eğitim ve test setlerine ayırıyoruz.
Verinin %80’ini eğitim, %20’sini test için kullanacağız.

Bu işlemi iki ayrı özellik kümesi için de yapacağız:
- Versiyon 1: `text + title` özellikleri
- Versiyon 2: `text + title + subject` özellikleri

In [ ]:
from sklearn.model_selection import train_test_split

#versiyon 1 (subject HARİÇ)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.2, random_state=42)

#versiyon 2 (subject DAHİL)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_with_subject, y, test_size=0.2, random_state=42)

## 14. Logistic Regression ile Model Eğitimi ve Karşılaştırma

İlk model olarak Logistic Regression seçtik. Bu model, metin sınıflandırma problemlerinde hızlı ve etkili sonuç vermesiyle bilinir.
Modeli, iki farklı veri seti üzerinde ayrı ayrı eğitiyoruz:
- Versiyon 1: `text + title` (TF-IDF özellikleri)
- Versiyon 2: `text + title + subject` (TF-IDF + kategorik veri)

Modellerin başarıları, doğruluk (accuracy), kesinlik (precision), duyarlılık (recall) ve F1 skoru gibi metriklerle karşılaştıracağız:       
**Accuracy (Doğruluk):** Modelin ne kadar doğru tahmin yaptığını gösterir. Ne kadar yüksekse o kadar doğru tahmin yaptığını gösterir ancak her zaman güvenilir değildir. Sınıflar (fake/real) dengesizse yanıltıcı olabilir, sınıfların sayısı birbirine olabildiğince yakın olmalıdır.     
**Precision (Kesinlik):** Modelin ‘sahte haber’ dediği şeylerin gerçekten ne kadar sahte olduğunu gösterir. Yani precision düşük ise, real olan haberleri fake sanma durumu çok fazladır. Bu da haksız suçlamalara ve iftiralara neden olabilir. Precision mümkün olduğunca yüksek olmalıdır.    
**Recall (Duyarlılık / Yakalama Oranı):** Gerçekten sahte olan haberlerin ne kadarını modelin yakalayabildiğini gösterir. Recall oranı, ne kadar düşük ise model o kadar fazla fake haberi gözden kaçırıyor demektir.    
**F1 Score (Denge Skoru):** Precision ve Recall’u dengede tutan ortak bir skordur. Modelin hem yanlış alarmları düşük olsun (precision yüksek), hem de fake haberleri gözden kaçırmasın (recall yüksek) istiyorsak F1 skoru ideal göstergedir. *F1 = 2 × (Precision × Recall) / (Precision + Recall)*

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def train_and_evaluate(X_train, X_test, y_train, y_test, label):
    print(f"--- {label} ---")
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

#versiyon 1 (subject HARİÇ)
train_and_evaluate(X_train_1, X_test_1, y_train_1, y_test_1, "Versiyon 1 (subject HARİÇ)")

#versiyon 2 (subject DAHİL)
train_and_evaluate(X_train_2, X_test_2, y_train_2, y_test_2, "Versiyon 2 (subject DAHİL)")

## 15. Model Değerlendirme Sonuçları ve Veri Sızıntısı Analizi

Logistic Regression modeli iki farklı özellik kümesi ile eğitildi ve test edildi:

### Versiyon 1 – subject HARİÇ (Yalnızca text ve title):

- **Accuracy:** 99.25%
- **Precision:** 99.33%
- **Recall:** 99.20%
- **F1 Score:** 99.26%

Bu versiyonda model, yalnızca haber metni (text) ve başlığını (title) kullanarak yüksek başarı sağlamıştır. Bu da haberlerdeki kelime örüntülerinin sahte ve gerçek haberleri ayırt etmekte oldukça etkili olduğunu göstermektedir.


### Versiyon 2 – subject DAHİL:

- **Accuracy:** 100%
- **Precision / Recall / F1:** 100%

Bu sonuç ilk bakışta mükemmel görünse de, yapılan ek analiz sonucunda modelin "subject" sütunundan **etiketi doğrudan ezberlediği** tespit edilmiştir. Aşağıdaki analiz ile `subject` sütunundaki her değerin yalnızca tek bir sınıfı temsil ettiği görülmüştür, bu modelin tahmin yapmak yerine direkt olarak ezberlediğinin kanıtıdır.

Bu aşamadan sonra **Versiyon 1** üzerinden devam edeceğiz, çünkü "subject" sütununu kullanmak modelimizi olumsuz etkiliyor.

In [ ]:
#orijinal dosyaları tekrar yükle
fake = pd.read_csv("/kaggle/input/real-and-fake-news/Fake.csv")
real = pd.read_csv("/kaggle/input/real-and-fake-news/True.csv")

fake['label'] = 1 #fake = 1
real['label'] = 0 #real = 0

combined = pd.concat([fake, real], ignore_index=True) #birleştir

#subject-label ilişki tablosu
pd.crosstab(combined['subject'], combined['label'])


## 15. Genellenebilirlik ve Ezberleme Riski (Overfitting) İncelemesi
Modelin eğitim ve test başarıları arasında çok küçük bir fark (yaklaşık %0.4) bulunuyor.
Bu, modelin eğitim verisini ezberlemek yerine anlamlı örüntüler öğrendiğini ve overfitting yapmadığını gösteriyor.         
Yani model, test verilerinde de tutarlı şekilde başarılı tahminler yapabildi. 
Bu nedenle nihai modelin gerçek dünyada genellenebilirlik açısından güvenli olduğu düşünüyorum.

In [ ]:
#eğitim ve test skorlarını karşılaştırmak için modeli yeniden eğitiyoruz
model = LogisticRegression(max_iter=300)
model.fit(X_train_1, y_train_1)

#eğitim seti performansı
y_train_pred = model.predict(X_train_1)
print("EĞİTİM SETİ:")
print("Accuracy:", accuracy_score(y_train_1, y_train_pred))
print("F1 Score:", f1_score(y_train_1, y_train_pred))

#test seti performansı
y_test_pred = model.predict(X_test_1)
print("\nTEST SETİ:")
print("Accuracy:", accuracy_score(y_test_1, y_test_pred))
print("F1 Score:", f1_score(y_test_1, y_test_pred))


## 16. Alternatif Model: Multinomial Naive Bayes

Logistic Regression modeliyle elde edilen başarıyı karşılaştırmak amacıyla, metin sınıflandırmada yaygın olarak kullanılan bir diğer model olan **Multinomial Naive Bayes** uygulayacağız.
Bu model, genellikle metin sınıflandırmada hızlı ve etkili sonuç verir ancak kelime örüntülerine dair daha yüzeysel bir analiz yapar.

### Naive Bayes Performansı:
Multinomial Naive Bayes modeli, TF-IDF ile sayısallaştırılmış "text" ve "title" verileri üzerinde eğittik.
- *Accuracy:* 94.86%
- *Precision:* 93.96%
- *Recall:* 96.00%
- *F1 Score:* 94.97%

### Karşılaştırma:
- Logistic Regression modeli %99.25 doğruluk ile daha başarılı bir performans gösterdi.
- Naive Bayes daha basit ve hızlı bir alternatif ancak **Logistic Regression** daha tutarlı sonuçlar verdi.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train_1, y_train_1)

y_pred_nb = nb_model.predict(X_test_1)

print("Naive Bayes - TEST SETİ")
print("Accuracy:", accuracy_score(y_test_1, y_pred_nb))
print("Precision:", precision_score(y_test_1, y_pred_nb))
print("Recall:", recall_score(y_test_1, y_pred_nb))
print("F1 Score:", f1_score(y_test_1, y_pred_nb))
print("\nClassification Report:\n", classification_report(y_test_1, y_pred_nb))

## 17. Alternatif Model – Support Vector Machine (SVM)

**Destek Vektör Makineleri (SVM)**, yüksek boyutlu veri ile çalışmada etkili bir sınıflandırma yöntemidir.  

TF-IDF ile elde edilen "text" ve "title" özellikleri ile SVM modeli eğitip, test ediyoruz.
Amaç: Logistic Regression modeli ile karşılaştırmalı bir değerlendirme yapmak.

Model, Logistic Regression ile benzer şekilde çok yüksek performans gösterdi:

- *Accuracy:* 99.54%
- *Precision:* 99.51%
- *Recall:* 99.58%
- *F1 Score:* 99.54%

### Karşılaştırma:

| Model                 | Accuracy | F1 Score |
|-----------------------|----------|----------|
| Logistic Regression   | 99.25%   | 99.26%   |
| SVM                   | 99.54%   | 99.54%   |
| Naive Bayes           | 94.86%   | 94.97%   |

In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(X_train_1, y_train_1)

y_pred_svm = svm_model.predict(X_test_1)

print("SVM - TEST SETİ")
print("Accuracy:", accuracy_score(y_test_1, y_pred_svm))
print("Precision:", precision_score(y_test_1, y_pred_svm))
print("Recall:", recall_score(y_test_1, y_pred_svm))
print("F1 Score:", f1_score(y_test_1, y_pred_svm))
print("\nClassification Report:\n", classification_report(y_test_1, y_pred_svm))

## 18. SVM Modeli Overfitting Kontrolü – Eğitim/Test Performansı Karşılaştırması

Modelin overfitting (aşırı öğrenme) yapıp yapmadığını anlamak için eğitim ve test seti üzerindeki performansları karşılaştıracağız:

| Set       | Accuracy | F1 Score |
|-----------|----------|----------|
| Eğitim    | 99.99%   | 99.99%   |
| Test      | 99.54%   | 99.54%   |
| Fark      | ~0.45%   | ~0.45%   |

Bu fark çok küçük olduğu için **overfitting durumu gözlemlenmemektedir**, modelin genelleme başarısı yüksektir.

In [ ]:
y_train_pred_svm = svm_model.predict(X_train_1)
print("EĞİTİM SETİ:")
print("Accuracy:", accuracy_score(y_train_1, y_train_pred_svm))
print("F1 Score:", f1_score(y_train_1, y_train_pred_svm))

## 19. Proje Sonucu ve Genel Değerlendirme

Bu projede, sahte ve gerçek haberleri ayırt edebilen bir makine öğrenmesi modeli geliştirilmiştir.  
Kullanılan veri seti, Kaggle üzerinde yer alan **"Real and Fake News"** veri setidir ve toplamda 44.000'den fazla haber içermektedir.

### Uygulanan Süreç:
- Veri temizleme ve eksik veri analizi yapıldı.
- *"text"*, *"title"* sütunları TF-IDF ile sayısallaştırıldı.
- *"subject"* sütunu başlangıçta modele dahil edilse de, **etiketle doğrudan ilişkili** olduğu (veri sızıntısı) tespit edilerek çıkarıldı.
- Veri **"train_test_split"** ile eğitim ve test kümelerine ayrıldı

### Kullanılan Modeller:
1. **Logistic Regression**  
2. **Naive Bayes**
3. **Support Vector Machine (SVM)**

### Model Performans Karşılaştırması (Test Seti):

| Model               | Accuracy | F1 Score |
|---------------------|----------|----------|
| Logistic Regression | 99.25%   | 99.26%   |
| SVM                 | 99.54%   | 99.54%   |
| Naive Bayes         | 94.86%   | 94.97%   |